<a href="https://colab.research.google.com/github/Park-da-in/ComputerVision_Seminar/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/CV_seminar_project

/content/drive/MyDrive/CV_seminar_project


In [3]:
%pwd

'/content/drive/MyDrive/CV_seminar_project'

In [4]:
%ls

 computer_vision_2주차.ipynb
'CV seminar_week2 과제풀이_실습.ipynb'
'CV seminar_week3_과제.ipynb'
'CV seminar_week3_과제_연습.ipynb'
'CV seminar_week7_epochs100.ipynb'
 dataset.py
 ex1.py
 modeling.ipynb
 model_name.pth
 original/
 __pycache__/
 resnet50.pth
 test/
 testing.ipynb
 train/
 valid/
 week4_과제.ipynb
'week4_실습2_파이토치 데이터셋 클래스커스텀.ipynb'
'week4_실습_파이토치 데이터셋 클래스.ipynb의 사본'


#1. 학습한 가중치 불러오기

In [5]:
import torch
state_dict = torch.load('resnet50.pth')

In [6]:
#best_epoch = state_dict['epoch']
best_test_acc = state_dict['test_acc']
weights = state_dict['net']

print(f'{10}번째 에포크에서 test셋 기준으로 {best_test_acc}를 달성하였습니다.')

10번째 에포크에서 test셋 기준으로 91.5를 달성하였습니다.


#2. 학습한 가중치를 모델에 적용하기기

In [7]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # device 배정
torch.manual_seed(42)
if device == 'cuda':
  torch.cuda.manual_seed_all(42)
device

'cuda'

In [8]:
from torchvision import models # 모델 라이브러리 함수

resnet_50 = models.resnet50(pretrained=True).to(device) # 선행학습 여부

# finetuning
import torch.nn as nn # 파이토치 뉴럴네트워크 layer 라이브러리
resnet_50.fc = nn.Linear(resnet_50.fc.in_features, 3).to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
# 학습한 가중치 적용완료
resnet_50.load_state_dict(weights)

<All keys matched successfully>

#3. testset의 최종 성능 확인하기
과제 
> 모델은 100epochs를 돌려서 만든 가중치를 이용한다.      
> test_loader를 생성하여, test set의 최종 성능을 평가한다.   
> 텐서보드는 자율로 이용하시오.


In [11]:
from dataset import Custom_dataset as C
from torch.utils.data import Dataset, DataLoader
import cv2
import os 
import torch
import torchvision
from torchvision import transforms # 이미지 데이터 augmentation
import glob
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 # albumentations 텐서화 함수

root_path = '/content/drive/MyDrive/CV_seminar_project'

train_transforms = A.Compose([
    A.Resize(224,224),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    A.ChannelShuffle(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 이미지넷 데이터셋 통계값으로 Normalize
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 텐서타입은 안해줌
    ToTensorV2() # Normalize를 먼저하고 tensor화를 진행해야한다.
])

### Pytorch 데이터 클래스 생성
test_class = C(root_path=root_path, mode='test', transforms=test_transforms)

### Pytorch BatchLoader 생성 (학습에 이용할 최종 dataloader)
from torch.utils.data import DataLoader as DataLoader

test_loader = DataLoader(test_class, batch_size=8, shuffle = False, num_workers=0)

In [12]:
resnet_50.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[ 1.2016e-02,  1.3540e-02, -1.6002e-02,  ..., -4.0960e-02,
                         -4.2534e-02, -7.0131e-02],
                        [ 2.3446e-03,  4.2468e-03,  1.3576e-02,  ...,  1.5090e-03,
                         -2.0857e-02, -3.8244e-02],
                        [ 2.0479e-02,  2.1910e-02,  1.4677e-02,  ...,  1.0188e-01,
                          6.2568e-02,  5.2147e-02],
                        ...,
                        [-1.8521e-03,  2.6806e-02, -1.0915e-02,  ..., -1.2807e-01,
                         -7.7011e-02,  7.5913e-03],
                        [ 2.7327e-03,  4.6688e-02,  6.0844e-02,  ...,  2.3215e-02,
                         -3.4210e-02, -1.5985e-02],
                        [-8.0649e-02, -3.3358e-02, -1.8813e-02,  ...,  3.4398e-02,
                          2.1922e-02,  1.3294e-03]],
              
                       [[-1.8202e-02,  1.1961e-02,  2.4741e-02,  ...,  5.5172e-02,
                          4.5335

In [18]:
test_acc = 0.0
total = 0
correct = 0

resnet_50.eval()
with torch.no_grad():
  for ii, (test_img, test_label) in enumerate(test_loader):
        # gpu에 할당
    test_img = test_img.to(device)
    test_label = test_label.to(device)

    output = resnet_50(test_img) # 모델에 입력

    # loss & acc
    _, predictions = torch.max(output.data ,dim = 1 )
    
    total += test_label.size(0)
    correct += (predictions == test_label).sum().item()
    test_acc += 100 * (correct/total)
result = test_acc/(ii+1)

In [23]:
print(f'학습한 모델의 최종 성능이 testset기준 {result:.2f}%를 기록하였습니다.')

학습한 모델의 최종 성능이 testset기준 91.50%를 기록하였습니다.
